# Coffee Shop Location Optimization Project
Technologies Used: Python, IBM CPLEX, Docplex, Geopy, Folium, Pandas

Project Overview: Optimized the locations of coffee shops within Chicago by minimizing the total distance to public libraries. The project used Mixed-Integer Linear Programming (MILP) to determine optimal coffee shop placements and assignments based on geospatial data.

Key Responsibilities:

Data Collection: Collected and processed geographic data of public libraries from the City of Chicago's open data portal (latitude, longitude).
Optimization Model: Formulated and solved a MILP using IBM CPLEX and Docplex to determine which libraries would serve as coffee shop locations, while minimizing the total distance between libraries and coffee shops.
Constraints Setup:
Implemented binary variables to represent the selection of coffee shop locations.
Applied constraints to ensure each library was assigned to exactly one coffee shop.
Managed fixed capacity constraints to ensure only a specified number of coffee shops were opened.
Solver Integration: Utilized IBM CPLEX’s optimization engine, overcoming size limitations in the free version to solve problems with fewer than 1000 variables and constraints.
Data Visualization: Visualized the library locations and selected coffee shops on an interactive map using Folium.
Challenge: Encountered solver size limits (1000 variables/constraints) due to the size of the problem, requiring adjustments to the problem structure.

Results:

Successfully determined optimal locations for coffee shops to minimize travel distances to libraries.
Visualized the model output on an interactive map for clear, data-driven decision-making.

Key Learnings:

Gained expertise in solving large-scale optimization problems using Docplex and IBM CPLEX.
Acquired experience with geospatial data handling and visualization in Python.
Learned the importance of solver limitations and the considerations for scaling optimization models.
You can adjust the language or format depending on the space available and the resume style you're using, but this should provide a concise and clear description of the project.

In [ ]:
import sys
import docplex.mp

In [ ]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

In [ ]:
!pip3 install docplex
!pip3 install cplex
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
try:
    import geopy.distance
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip3 install geopy
    else:
        !pip3 install --user geopy

In [ ]:
!pip3 install docplex

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# Simple distance computation between 2 locations.
from geopy.distance import great_circle

def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

In [ ]:
def build_libraries_from_url(url, name_pos, lat_long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[name_pos]
        try:

            latitude = float(location[lat_long_pos][1])
            longitude = float(location[lat_long_pos][2])
        except TypeError:
            latitude = longitude = None
        try:
            name = str(uname)
        except:
            name = "???"
        name = "P_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

In [ ]:
libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD',
                                   name_pos=10,
                                   lat_long_pos=17)

In [ ]:
print("There are %d public libraries in Chicago" % (len(libraries)))

There are 81 public libraries in Chicago


In [ ]:
nb_shops = 3
print("We would like to open %d coffee shops" % nb_shops)

We would like to open 3 coffee shops


In [ ]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip3 install folium
    else:
        !pip3 install folium

In [ ]:
import folium

max_distance = 5.8 #adjusted to fit CPLEX model constraints
central_location = NamedPoint("Chicago_Center",  -87.629, 41.878)

filtered_libraries = [library for library in libraries if get_distance(central_location, library) <= max_distance]
libraries = filtered_libraries

print(len(libraries))
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

30


In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Darwin 64bit
* Python version 3.9.6, located at: /Library/Developer/CommandLineTools/usr/bin/python3
* docplex is present, version is 2.29.241
* CPLEX library is present, version is 22.1.1.0, located at: /Users/starz/Library/Python/3.9/lib/python/site-packages
* pandas is present, version is 2.2.3


In [ ]:
from docplex.mp.model import Model

mdl = Model("coffee shops")

In [ ]:
BIGNUM = 999999999

# Ensure unique points
libraries = set(libraries)
# For simplicity, let's consider that coffee shops candidate locations are the same as libraries locations.
# That is: any library location can also be selected as a coffee shop.
coffeeshop_locations = libraries

# Decision vars
# Binary vars indicating which coffee shop locations will be actually selected
coffeeshop_vars = mdl.binary_var_dict(coffeeshop_locations, name="is_coffeeshop")
#
# Binary vars representing the "assigned" libraries for each coffee shop
link_vars = mdl.binary_var_matrix(coffeeshop_locations, libraries, "link")

In [ ]:
for c_loc in coffeeshop_locations:
    for b in libraries:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl.add_constraint(link_vars[c_loc, b] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

In [ ]:
mdl.add_constraints(link_vars[c_loc, b] <= coffeeshop_vars[c_loc]
                   for b in libraries
                   for c_loc in coffeeshop_locations)
mdl.print_information()

Model: coffee shops
 - number of variables: 930
   - binary=930, integer=0, continuous=0
 - number of constraints: 900
   - linear=900
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [ ]:
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1
                   for b in libraries)
mdl.print_information()

Model: coffee shops
 - number of variables: 930
   - binary=930, integer=0, continuous=0
 - number of constraints: 930
   - linear=930
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [ ]:
# Total nb of open coffee shops
mdl.add_constraint(mdl.sum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

Model: coffee shops
 - number of variables: 930
   - binary=930, integer=0, continuous=0
 - number of constraints: 931
   - linear=931
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [ ]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in coffeeshop_locations for b in libraries)
mdl.minimize(total_distance)

In [ ]:
print("# coffee shops locations = %d" % len(coffeeshop_locations))
print("# coffee shops           = %d" % nb_shops)

assert mdl.solve(), "!!! Solve of the model fails"

# coffee shops locations = 30
# coffee shops           = 3


In [ ]:
total_distance = mdl.objective_value
open_coffeeshops = [c_loc for c_loc in coffeeshop_locations if coffeeshop_vars[c_loc].solution_value == 1]
not_coffeeshops = [c_loc for c_loc in coffeeshop_locations if c_loc not in open_coffeeshops]
edges = [(c_loc, b) for b in libraries for c_loc in coffeeshop_locations if int(link_vars[c_loc, b]) == 1]

print("Total distance = %g" % total_distance)
print("# coffee shops  = {0}".format(len(open_coffeeshops)))
for c in open_coffeeshops:
    print("new coffee shop: {0!s}".format(c))

Total distance = 53.3444
# coffee shops  = 3
new coffee shop: P_642 W. 43rd St._11
new coffee shop: P_1150 W. Fullerton Ave._32
new coffee shop: P_6 S. Hoyne Ave._13


In [ ]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for coffeeshop in open_coffeeshops:
    lt = coffeeshop.y
    lg = coffeeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)

for b in libraries:
    if b not in open_coffeeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(map_osm)


for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_child(folium.PolyLine(coordinates, color='#FF0000', weight=5))

map_osm